In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, Slider, CustomJS
from bokeh.layouts import column
import pandas as pd

# 1️⃣ Leer datasets
asthma = pd.read_csv("NYC EH Data Portal - Adults with asthma (past 12 months) (full table).csv")
pm25 = pd.read_csv("NYC EH Data Portal - Fine particles (PM 2.5) (full table).csv")

# 2️⃣ Clean columns (remove '*' and convert to numeric)
asthma['Percent_clean'] = asthma['Percent'].str.extract(r'(\d+\.\d+)').astype(float)
asthma['TimePeriod'] = asthma['TimePeriod'].astype(int)

pm25['Annual_mean'] = pm25['Annual mean mcg/m3']
pm25['TimePeriod'] = pm25['TimePeriod'].astype(int)

# 3️⃣ Group both by year (mean)
asthma_yearly = asthma.groupby('TimePeriod')['Percent_clean'].mean().reset_index()
pm25_yearly = pm25.groupby('TimePeriod')['Annual_mean'].mean().reset_index()

# 4️⃣ Normalize both series (Min-Max scaling)
asthma_yearly['Percent_norm'] = (asthma_yearly['Percent_clean'] - asthma_yearly['Percent_clean'].min()) / (asthma_yearly['Percent_clean'].max() - asthma_yearly['Percent_clean'].min())
pm25_yearly['Annual_norm'] = (pm25_yearly['Annual_mean'] - pm25_yearly['Annual_mean'].min()) / (pm25_yearly['Annual_mean'].max() - pm25_yearly['Annual_mean'].min())

# 5️⃣ Prepare ColumnDataSources
source_asthma = ColumnDataSource(asthma_yearly)  # Fuente original de Asma
source_pm25 = ColumnDataSource(pm25_yearly)

# 6️⃣ Fuente de datos desplazada para la línea de Asma (esto se moverá con el slider)
source_asthma_shifted = ColumnDataSource(data=dict(TimePeriod=asthma_yearly['TimePeriod'], Percent_norm=asthma_yearly['Percent_norm']))

# 7️⃣ Crear figura
p = figure(title='Normalized Asma vs PM2.5 with Temporal Shift',
           x_axis_label='Year',
           y_axis_label='Normalized Value (0-1)',
           width=900, height=500,
           tools='pan,wheel_zoom,box_zoom,reset,save')

# Asma → naranja (#ff7f0e) (Línea desplazable)
line1 = p.line('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', legend_label='Asma (Normalized)', line_width=3)
circle1 = p.circle('TimePeriod', 'Percent_norm', source=source_asthma_shifted, color='#ff7f0e', size=8)

# PM2.5 → morado (#9467bd)
line2 = p.line('TimePeriod', 'Annual_norm', source=source_pm25, color='#9467bd', legend_label='PM2.5 (Normalized)', line_width=3)
circle2 = p.circle('TimePeriod', 'Annual_norm', source=source_pm25, color='#9467bd', size=8)

# 8️⃣ Hover Tool para Asma (usar datos originales de Asma)
hover_asthma = HoverTool()
hover_asthma.tooltips = [
    ('Year', '@TimePeriod'),
    ('Asma Value', '@Percent_norm')
]
# Asociar el hover solo a la línea y círculos de Asma desplazada
hover_asthma.renderers = [line1, circle1]
p.add_tools(hover_asthma)

# 9️⃣ Hover Tool para PM2.5 (usar datos originales de PM2.5)
hover_pm25 = HoverTool()
hover_pm25.tooltips = [
    ('Year', '@TimePeriod'),
    ('PM2.5 Value', '@Annual_norm')
]
# Asociar el hover solo a la línea y círculos de PM2.5
hover_pm25.renderers = [line2, circle2]
p.add_tools(hover_pm25)

# 10️⃣ Interactividad en el gráfico (desaparecer elementos al hacer clic)
p.legend.click_policy = "hide"

# 🔥 Agregar Slider para desplazar la línea de Asma
shift_slider = Slider(start=-5, end=5, value=0, step=1, title="Shift Asma Curve (Years)")

# Definir la función de desplazamiento con CustomJS (no modificar la fuente original)
callback = CustomJS(args=dict(source=source_asthma_shifted, slider=shift_slider,
                               original_x=asthma_yearly['TimePeriod'].tolist(),
                               original_y=asthma_yearly['Percent_norm'].tolist()), code=""" 
    const data = source.data;
    const shift = slider.value;
    const x = original_x;
    const y = original_y;
    const new_x = [];
    for (let i = 0; i < x.length; i++) {
        new_x.push(x[i] + shift);
    }
    data['TimePeriod'] = new_x;
    source.change.emit();
""")

shift_slider.js_on_change('value', callback)

# 🔥 Mostrar gráfico con el slider
show(column(p, shift_slider))


KeyError: 'Percent_norm'